# Simulated streamflow analysis
## Juncal Norte - august, 2022
#### Paul Sandoval Quilodrán - https://github.com/SQPaul

## Import packages 

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import re
import matplotlib.pyplot as plt
import datetime
import hydroeval as he

### Streamflow by source

In [2]:
#Define paths

rain_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\QRAINSubBasinTSS.tss"
snow_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\QSNOWSubBasinTSS.tss"
glacier_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\QGLACSubBasinTSS.tss"
baseflow_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\QBASFSubBasinTSS.tss"
obs_path = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Caudales\qobs_v1.xlsx"

paths = [rain_path, snow_path, glacier_path, baseflow_path]

rain = []
snow = []
glacier = []
baseflow = []
sim = []

streamflow = [rain,snow,glacier,baseflow]

for i in range(len(paths)):
    var = pd.read_csv(paths[i],index_col=False,skiprows=4)
    for j in range(len(var)):
        val = var.iloc[j,0].strip()
        val = " ".join(val.split())
        streamflow[i].append(val)       
    streamflow[i] = pd.DataFrame(streamflow[i])
    streamflow[i].columns = ["name"]
    streamflow[i] = streamflow[i]["name"].str.split(" ",expand=True)
    streamflow[i] = pd.DataFrame(streamflow[i].iloc[:,2])


obs = pd.read_excel(obs_path,sheet_name="4",engine="openpyxl")
        
rain = pd.DataFrame(streamflow[0]).astype(float)
rain.columns = ["rain"]
snow = pd.DataFrame(streamflow[1]).astype(float)
snow.columns = ["snow"]
glacier = pd.DataFrame(streamflow[2]).astype(float)
glacier.columns = ["glacier"]
baseflow = pd.DataFrame(streamflow[3]).astype(float)
baseflow.columns = ["baseflow"]

for k in range(len(rain)):
    sim.append(rain.iloc[k,0]+snow.iloc[k,0]+glacier.iloc[k,0]+baseflow.iloc[k,0])

sim = pd.DataFrame(sim)
sim.columns = ["sim"]
obs = pd.DataFrame(obs.iloc[:,1]).astype(float)
obs.columns = ["obs"]

streamflow = pd.concat([baseflow,rain,snow,glacier,sim,obs],axis=1)
streamflow.to_excel(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Caudales\postprocess\streamflow.xlsx")

In [5]:
streamflow

,baseflow,rain,snow,glacier,sim,obs
0,0.000000,0.000000,0.180000,0.014776,0.194776,11.30
1,0.000000,0.000000,0.177840,0.036118,0.213958,11.40
2,0.000000,0.000000,0.175706,0.049281,0.224987,10.80
3,0.000000,0.000000,0.173597,0.066418,0.240015,9.68
4,0.000000,0.000000,0.171514,0.086482,0.257996,9.98
...,...,...,...,...,...,...
4013,0.000008,0.000167,2.192310,0.059457,2.251941,9.64
4014,0.000007,0.000277,2.201560,0.066556,2.268400,8.68
4015,0.000007,0.000655,2.183290,0.068738,2.252691,7.70
4016,0.000007,0.001102,2.184840,0.069608,2.255557,6.23


#### Ploteo

In [ ]:
#glacier.plot(linestyle="-",color="seagreen")
#baseflow.iloc[0,0]

### Sensitivity analysis

#### KGE

In [6]:
#Import paths
obs_path = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Caudales\qobs_v1.xlsx"
sim_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\QTOTSubBasinTSS.tss"
thickness_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\ICE_DEPTH.csv"


#Read and prepare obs streamflow
obs = pd.read_excel(obs_path,sheet_name="4",engine="openpyxl")
obs = obs.iloc[:,1]

#Read and prepare sim streamflow
sim_read = pd.read_csv(sim_path,index_col=False,skiprows=4)
sim = []
for v in range(len(sim_read)):
    val = sim_read.iloc[v,0].strip()
    val = " ".join(val.split())
    sim.append(val)
sim = pd.DataFrame(sim)
sim.columns = ["name"]
sim = sim["name"].str.split(" ",expand=True)
sim = pd.DataFrame(sim.iloc[:,2]).astype(float)
sim = sim.iloc[:,0]

#Calculate KGE'
kge = pd.DataFrame(he.evaluator(he.kgeprime, sim, obs),index=["KGE","r","γ","β"])
kge

#Read thickness
thickness = pd.read_csv(thickness_path)

g_14053 = []
g_14088 = []
g_14092 = []
g_14093 = []

for i in range(len(thickness)):
        if thickness.iloc[i,0] == "2000-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2005-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2010-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2015-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2020-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])

glacier_t = pd.DataFrame(np.column_stack([g_14053,g_14088,g_14092,g_14093]), 
                               columns=["14053","14088","14092","14093"])
glacier_t.index = ["2000","2005","2010"]

with pd.ExcelWriter(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Calibration\Sensitivity_analysis\sensitivity_analysis.xlsx") as writer:
    kge.to_excel(writer, sheet_name='kge')
    glacier_t.to_excel(writer, sheet_name='thickness')
    
kge

,0
KGE,0.469743
r,0.818426
γ,1.071533
β,0.506962


In [7]:
sim

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
4018    1.42109
4019    1.64167
4020    1.78325
4021    1.79842
4022    1.83972
Name: 2, Length: 4023, dtype: float64

In [4]:
glacier_t

,14053,14088,14092,14093
2000,36.243946,54.231533,58.714203,30.789492
2005,21.219837,36.307260,41.129080,5.921966
2010,17.953270,30.262596,35.797090,0.000000


#### NASH

In [7]:
#Import paths
obs_path = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Caudales\qobs_v1.xlsx"
sim_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\QTOTSubBasinTSS.tss"
thickness_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\ICE_DEPTH.csv"


#Read and prepare obs streamflow
obs = pd.read_excel(obs_path,sheet_name="4",engine="openpyxl")
obs = obs.iloc[:,1]

#Read and prepare sim streamflow
sim_read = pd.read_csv(sim_path,index_col=False,skiprows=4)
sim = []
for v in range(len(sim_read)):
    val = sim_read.iloc[v,0].strip()
    val = " ".join(val.split())
    sim.append(val)
sim = pd.DataFrame(sim)
sim.columns = ["name"]
sim = sim["name"].str.split(" ",expand=True)
sim = pd.DataFrame(sim.iloc[:,2]).astype(float)
sim = sim.iloc[:,0]

#Calculate KGE'
nash = pd.DataFrame(he.evaluator(he.nse, sim, obs),index=["NASH"])
nash

#Read thickness
thickness = pd.read_csv(thickness_path)

g_14053 = []
g_14088 = []
g_14092 = []
g_14093 = []

for i in range(len(thickness)):
        if thickness.iloc[i,0] == "2000-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2005-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2010-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2015-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2020-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])

glacier_t = pd.DataFrame(np.column_stack([g_14053,g_14088,g_14092,g_14093]), 
                               columns=["14053","14088","14092","14093"])
glacier_t.index = ["2000","2005","2010"]
    
nash

,0
NASH,0.222838


In [6]:
glacier_t

,14053,14088,14092,14093
2000,36.243946,54.231533,58.714203,30.789492
2005,16.324545,33.753372,38.856880,1.697353
2010,13.794436,26.272562,31.836853,0.000000
